In [ ]:
import numpy as np
from lattice import Lattice2D, BrickwallLattice, SquareLattice

In [4]:
bw = BrickwallLattice((6, 12), 1, 1, np.array([1, 0]), 1e-3, T=12, initial_occupation=0.5)
bw.evolve(options={'progress_bar': True})

Creating Hamiltonians...
Done.
Calculating energy eigenstates...
Done.
50 % of states were set as initially occupied.
10.0%. Run time:  10.01s. Est. time left: 00:00:01:30
20.0%. Run time:  17.68s. Est. time left: 00:00:01:10
30.0%. Run time:  25.56s. Est. time left: 00:00:00:59
40.0%. Run time:  33.25s. Est. time left: 00:00:00:49
50.0%. Run time:  41.11s. Est. time left: 00:00:00:41
60.0%. Run time:  48.82s. Est. time left: 00:00:00:32
70.0%. Run time:  56.75s. Est. time left: 00:00:00:24
80.0%. Run time:  64.45s. Est. time left: 00:00:00:16
90.0%. Run time:  72.33s. Est. time left: 00:00:00:08
100.0%. Run time:  80.06s. Est. time left: 00:00:00:00
Total run time:  80.06s


In [5]:
max_c = max([max(bw.curl(bw.get_current_density(state)).values()) for state in bw.states])
bw.save_current_density_animation('bw_lattice-1.gif', sample_every=97, curl_norm=max_c, fps=15,)

Generating animation:  99%|█████████▉| 122/123 [00:38<00:00,  3.13frame/s]


In [6]:
l = Lattice2D((6, 12), 1, 1, np.array([1, 0]), 1e-3, T=12, initial_occupation=0.5)
l.evolve(options={"progress_bar":True})

Creating Hamiltonians...
Done.
Calculating energy eigenstates...
Done.
50 % of states were set as initially occupied.
10.0%. Run time:   9.09s. Est. time left: 00:00:01:21
20.0%. Run time:  16.34s. Est. time left: 00:00:01:05
30.0%. Run time:  23.78s. Est. time left: 00:00:00:55
40.0%. Run time:  31.03s. Est. time left: 00:00:00:46
50.0%. Run time:  38.15s. Est. time left: 00:00:00:38
60.0%. Run time:  45.14s. Est. time left: 00:00:00:30
70.0%. Run time:  52.18s. Est. time left: 00:00:00:22
80.0%. Run time:  59.12s. Est. time left: 00:00:00:14
90.0%. Run time:  66.21s. Est. time left: 00:00:00:07
100.0%. Run time:  73.43s. Est. time left: 00:00:00:00
Total run time:  73.43s


In [9]:
max_c = max([max(bw.curl(bw.get_current_density(state)).values()) for state in bw.states])
l.save_current_density_animation("lattice-8.gif", curl_norm=max_c, sample_every=97, fps=15)

Generating animation:  99%|█████████▉| 122/123 [00:45<00:00,  2.66frame/s]
